In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--window-size=1280,800")

driver = webdriver.Chrome(options=options)

try:
    driver.get("https://matchcenter.fvrz.ch/default.aspx?oid=11&lng=1&ln=&v=0&tg=4066900")
    html = driver.page_source
    soupSel = BeautifulSoup(html, "html.parser")
    print(soupSel)
finally:
    driver.quit()

In [10]:
# making a request to a webpage
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
}
url = "https://matchcenter.fvrz.ch/default.aspx?oid=11&lng=1&v=691581&t=66332&sg=68264&ls=24701&tg=4182972"
page = requests.get(url, headers=headers, timeout=2)
print(page.status_code)
print(page.content[:10000])

200
b'\r\n<!DOCTYPE html>\r\n<html lang="de">\r\n<head>\r\n    <title id="PageTitle">Fussballverband Region Z&#252;rich - Match center</title>\r\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />\r\n    <meta name="format-detection" content="telephone=no" />\r\n    <meta name="robots" content="index,follow" >\r\n    \r\n<link href="/include_2023/assets/plugins/owl-carousel/dist/assets/owl.theme.default.min.css" type="text/css" rel="stylesheet" media="all" />\r\n<link href="/include_2023/assets/plugins/owl-carousel/dist/assets/owl.carousel.min.css" type="text/css" rel="stylesheet" media="all" />\r\n<link href="/include_2023/assets/plugins/bootstrap/css/bootstrap.min.css" type="text/css" rel="stylesheet" media="all" />\r\n<link href="/include_2023/assets/plugins/bootstrap-icons/bootstrap-icons.css" type="text/css" rel="stylesheet" media="all" />\r\n<link href="/include_

In [13]:
soup = BeautifulSoup(page.content, "html.parser") # converts the page content into a beautifulsoup object
print(f"Page status: {page.status_code}")
if page.status_code == 200:
    print("Successfully created BeautifulSoup object")

Page status: 200
Successfully created BeautifulSoup object


In [25]:
# i will save the html file.. 
with open("../data/pagina_cache.html", "w", encoding="utf-8") as f:
    f.write(str(soup))


In [ ]:
# read from the same path.. 
with open("../data/pagina_cache.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

In [12]:
soup.get_text()[:5000]

"\n\n\n\nFussballverband Region Zürich - Match center\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nVerein\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFR Soccer\n\n\n\n\nSpielbetrieb \n\nAktuelle Spiele\nResultate + Ranglisten\nVereinsspielplan\nHeimspielplan\nStrafenSuspensionen/Funktionssperren\n\n\n\nTeams \n\nYouth League C a\nJunioren C 1. Stärkeklasse b\nJunioren D-9 a\nJunioren D-9 b\nJunioren D-9 c\nJunioren D-7 a\nJunioren D-7 b\nJunioren D-7 c\nJunioren E a\nJunioren E b\nJunioren F a\nJunioren F b\n\n\n\nVerein \n\nVereinsfunktionäre\nTrainer/in\nSchiedsrichter/Spielleiter\nTenu-Sponsoren\nGruppierungen\nSpielplan download\nClub-Widget Konfiguration\n\n clubcorner.ch\n\n\n\nSportanlagen \n\nBuchholz\nHeusser-Staub-Wiese\n\n\n\n\n\n\n\n\n\n\n\n\n\nSeestrasse 1178610\xa0Usterwww.footrebel.comVereinsnr.:\xa011071\n\nTeams: 12Farben: pinkGrü

In [58]:
def drop_last_char(text, letter):
    if (text[-1]=="letter") and (len(text) > 0):
        return text[:-1]
    return text
# Buscar todos los div con la clase 'shortTeamHeim'
teamHeim = soup.find_all("div", class_="shortTeamHeim")

for e in teamHeim:
    print(e.text.strip())
teamGast = soup.find_all("div", class_="shortTeamGast")
for e in teamGast:
    print(e.text.strip())
print(type(teamHeim))

FR SOCCER a
FC Wiedikon ZH a
<class 'bs4.element.ResultSet'>


In [ ]:
resultados = {}  # final dictionary to store results

# Search all blocks on the local team
bloques = soup.find_all("div", class_="shortTeamHeimDiv")

for bloque in bloques:
    # Get the local team name
    equipo_tag = bloque.find("div", class_="shortTeamHeim")
    equipo = equipo_tag.text.strip() if equipo_tag else "Desconocido"

    goles = []

    # Find all rows in the table
    filas = bloque.select("table tr")
    for fila in filas:
        minuto_tag = fila.find("td", class_="shortTimeHome")
        jugador_tag = fila.find("td", class_="shortSpielerHome")

        if minuto_tag and jugador_tag:
            minuto = minuto_tag.text.strip().replace("'", "")
            jugador = jugador_tag.text.strip()
            goles.append({"minuto": minuto, "jugador": jugador})

    resultados[equipo] = goles

print(resultados)

{'FR SOCCER a': [{'minuto': '55', 'jugador': 'Evan Agoume'}, {'minuto': '60', 'jugador': 'Mikail Mislimi'}, {'minuto': '83', 'jugador': 'Mikail Mislimi'}], 'Desconocido': []}


In [44]:
# to know the Game Number ID
game_id_tag = soup.find("small", class_="shortSpielort")
game_id = game_id_tag.text.strip() if game_id_tag else "Unknown"
#Meisterschaft - Youth League C / Herbstrunde / Gruppe  3 - 25.10.2025 16:00 - 
game_number = game_id.split("Spielnummer: ")[1].split()[0]
league = game_id.split("/")[0].split("-")[1]
group = game_id.split("/")[2].split("-")[0]
print(game_number, league, group)

132406  Youth League C   Gruppe  3 


In [77]:

data = []  # lista de dicts (una fila por gol)

# ---- TEAM LOCAL ---- here i will extract the data
# bloque is block
# equipo is team
# fila is row
# minuto is minute
# jugador is player

for bloque in soup.find_all("div", class_="shortTeamHeimDiv"):
    equipo_tag = bloque.find("div", class_="shortTeamHeim")
    equipo = equipo_tag.text.strip() if equipo_tag else "Unknown"

    for fila in bloque.select("table tr"):
        minuto_tag = fila.find("td", class_="shortTimeHome")
        jugador_tag = fila.find("td", class_="shortSpielerHome")

        if minuto_tag and jugador_tag:
            data.append({
                "team": equipo,
                "player": jugador_tag.text.strip(),
                "event": "goal",
                "min": minuto_tag.text.strip().replace("'", ""),
                "game_number": game_number,
                "league": league,   
                "group": group
            })

# ---- GUEST TEAM ----
for bloque in soup.find_all("div", class_="shortTeamHeimDiv"):
    equipo_tag = bloque.find("div", class_="shortTeamGast")
    equipo = equipo_tag.text.strip() if equipo_tag else "Unknown"

    for fila in bloque.select("table tr"):
        minuto_tag = fila.find("td", class_="shortTimeGast")
        jugador_tag = fila.find("td", class_="shortSpielerGast")

        if minuto_tag and jugador_tag:
            data.append({
                "team": equipo,
                "player": jugador_tag.text.strip(),
                "event": "goal",
                "min": minuto_tag.text.strip().replace("'", ""),
                "game_number": game_number,
                "league": league,   
                "group": group
            })

# ---- Converted DataFrame ----
df = pd.DataFrame(data)
#cleaning team names that end with " a"
df['team'] = df['team'].apply(lambda x: " ".join(x.split()[:-1]) if x.strip().endswith(" a") else x.strip())

df

,team,player,event,min,game_number,league,group
0,FR SOCCER,Evan Agoume,goal,55,132406,Youth League C,Gruppe 3
1,FR SOCCER,Mikail Mislimi,goal,60,132406,Youth League C,Gruppe 3
2,FR SOCCER,Mikail Mislimi,goal,83,132406,Youth League C,Gruppe 3
3,FC Wiedikon ZH,Livio Corbat,goal,22,132406,Youth League C,Gruppe 3
4,FC Wiedikon ZH,Laurin Nadig,goal,31,132406,Youth League C,Gruppe 3
5,FC Wiedikon ZH,Pablo Paris Quiceno,goal,34,132406,Youth League C,Gruppe 3
6,FC Wiedikon ZH,Enea Trippetta,goal,45,132406,Youth League C,Gruppe 3
7,FC Wiedikon ZH,Jayden Amstalden,goal,79,132406,Youth League C,Gruppe 3
8,FC Wiedikon ZH,Enea Trippetta,goal,84,132406,Youth League C,Gruppe 3


In [ ]:
kartes = ["Verwarnung", "Rot-Karte"]

# Encontrar el contenedor correcto
container = soup.select_one('div[id*="VereinMasterObjectID"][id$="pnlEvents"]')

# Todos los <li> dentro del timeline
events = container.select('ul.bnEventsList > li')

tarjetas = []  # lista para guardar todos los eventos de amonestación

for evento in events:
    time_tag = evento.find("time", class_="timeline-time")
    label_tag = evento.find("div", class_="eventlabel")

    if not label_tag:
        continue

    texto = label_tag.text.strip()

    if texto.startswith("Verwarnung"):  # sólo los de tarjeta amarilla
        minuto = time_tag.text.strip().replace("'", "") if time_tag else None

        # eliminar la palabra "Verwarnung"
        texto = texto.replace("Verwarnung", "").strip()

        # dividir por "(" para separar nombre y equipo
        partes = texto.split("(")
        jugador = partes[0].strip()  # antes del paréntesis
        equipo = partes[1].replace(")", "").strip() if len(partes) > 1 else None

        tarjetas.append({
            "team": equipo.upper(),
            "player": jugador,
            "event":"Yellow Card",
            "min": minuto,
            "game_number": game_number,
            "league": league,   
            "group": group
            
        })

# Mostrar el resultado
df2 = pd.DataFrame(tarjetas)

dff = pd.concat([df, df2], ignore_index=True)

columnas = dff.columns.tolist()
print(columnas)
dff

['team', 'player', 'event', 'min', 'game_number', 'league', 'group']


,team,player,event,min,game_number,league,group
0,FR SOCCER,Evan Agoume,goal,55,132406,Youth League C,Gruppe 3
1,FR SOCCER,Mikail Mislimi,goal,60,132406,Youth League C,Gruppe 3
2,FR SOCCER,Mikail Mislimi,goal,83,132406,Youth League C,Gruppe 3
3,FC Wiedikon ZH,Livio Corbat,goal,22,132406,Youth League C,Gruppe 3
4,FC Wiedikon ZH,Laurin Nadig,goal,31,132406,Youth League C,Gruppe 3
5,FC Wiedikon ZH,Pablo Paris Quiceno,goal,34,132406,Youth League C,Gruppe 3
6,FC Wiedikon ZH,Enea Trippetta,goal,45,132406,Youth League C,Gruppe 3
7,FC Wiedikon ZH,Jayden Amstalden,goal,79,132406,Youth League C,Gruppe 3
8,FC Wiedikon ZH,Enea Trippetta,goal,84,132406,Youth League C,Gruppe 3
9,FR SOCCER,Leonard Saku,Yellow Card,90,132406,Youth League C,Gruppe 3


In [96]:
import yaml
# Save it in a yaml file
with open("../data/teamsgame.yaml", "w", encoding="utf-8") as f:
    yaml.dump(dff.to_dict(orient="records"), f, allow_unicode=True)

In [97]:
# read it back!
with open("../data/teamsgame.yaml", "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)

df_game = pd.DataFrame(data)
columnas =['team', 'player', 'event', 'min', 'game_number', 'league', 'group']

df_game = df_game[columnas]
df_game

,team,player,event,min,game_number,league,group
0,FR SOCCER,Evan Agoume,goal,55,132406,Youth League C,Gruppe 3
1,FR SOCCER,Mikail Mislimi,goal,60,132406,Youth League C,Gruppe 3
2,FR SOCCER,Mikail Mislimi,goal,83,132406,Youth League C,Gruppe 3
3,FC Wiedikon ZH,Livio Corbat,goal,22,132406,Youth League C,Gruppe 3
4,FC Wiedikon ZH,Laurin Nadig,goal,31,132406,Youth League C,Gruppe 3
5,FC Wiedikon ZH,Pablo Paris Quiceno,goal,34,132406,Youth League C,Gruppe 3
6,FC Wiedikon ZH,Enea Trippetta,goal,45,132406,Youth League C,Gruppe 3
7,FC Wiedikon ZH,Jayden Amstalden,goal,79,132406,Youth League C,Gruppe 3
8,FC Wiedikon ZH,Enea Trippetta,goal,84,132406,Youth League C,Gruppe 3
9,FR SOCCER,Leonard Saku,Yellow Card,90,132406,Youth League C,Gruppe 3
